In [1]:
!pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -i -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/accelerate.git

!pip install transformers==4.30
!pip install -q -U sentencepiece
!pip install tdqm torch>=0.3.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 MB 17.1 MB/s eta 0:00:00
  Installi

In [3]:
!pip install huggingface_hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
import os
import random
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset, Dataset
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

In [8]:
from datasets import get_dataset_split_names

dataset_name = 'CarperAI/openai_summarize_comparisons'
get_dataset_split_names(dataset_name)


['test', 'train', 'valid1', 'valid2']

In [9]:
dataset = load_dataset('CarperAI/openai_summarize_comparisons', split='valid1')

dataset.info.features

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/92534 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/83629 [00:00<?, ? examples/s]

Generating valid1 split:   0%|          | 0/33082 [00:00<?, ? examples/s]

Generating valid2 split:   0%|          | 0/50715 [00:00<?, ? examples/s]

{'prompt': Value(dtype='string', id=None),
 'chosen': Value(dtype='string', id=None),
 'rejected': Value(dtype='string', id=None)}

In [19]:
train_dict = {'prompt': [], 'response':[]}
eval_dict = {'prompt': [], 'response':[]}

shuffled_dataset = dataset.shuffle(seed=42)
for i in range(int(len(shuffled_dataset)/2)):
  if random.randint(1,2)==1:
    key = 'chosen'
  else:
    key = 'rejected'
  train_dict["prompt"].append(shuffled_dataset[i]["prompt"])
  train_dict["response"].append(shuffled_dataset[i][key])
  eval_dict["prompt"].append(shuffled_dataset[i]["prompt"])
  eval_dict["response"].append(shuffled_dataset[i][key])

train_dataset = Dataset.from_dict(train_dict)
eval_dataset =  Dataset.from_dict(eval_dict)

In [11]:
model_id = "t5-base"

In [12]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    use_cache=False,
    load_in_8bit=True,
    device_map="auto"
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [13]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(base_model)

trainable params: 81345792 || all params: 222903552 || trainable%: 36.49371724682072


In [ ]:
# see the available modules by printint out the model
print(base_model)

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          model_max_length=512,
                                          truncation=True,
                                          padding=True)
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

In [17]:
training_args = TrainingArguments(   # https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
    output_dir='./output/',
    # save_steps=5000,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=1e-3,
    logging_steps=500,
    optim='adamw_torch',
    # bf16=True,
    # remove_unused_columns=False,
    run_name="sft_t5_base",
)

In [20]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    dataset_text_field="response",
    max_seq_length=512
)
trainer.train()
trainer.save_model('./output/')

Map:   0%|          | 0/16541 [00:00<?, ? examples/s]

Map:   0%|          | 0/16541 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.272000
1000,0.000000
1500,0.000000
2000,0.000000
2500,0.000000
3000,0.000000
3500,0.000000
4000,0.000000
4500,0.000000
5000,0.000000


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1709: UserWarning: You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected behaviors. If you want to save 8-bit models, make sure to have `bitsandbytes>0.37.2` installed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1709: UserWarning: You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected behaviors. If you want to save 8-bit models, make sure to have `bitsandbytes>0.37.2` installed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1709: UserWarning: You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected behaviors. If you want to save 8-bit models, make sure to have `bitsandbytes>0.37.2` installed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1709: UserWarning: You are

In [21]:
trainer.model.push_to_hub('PanoEvJ/T5_base_SFT_summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'generation_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'pytorch_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/418M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/418M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PanoEvJ/T5_base_SFT_summarization/commit/083e20e9a50f81ca0f8d364450c486696799adcc', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='083e20e9a50f81ca0f8d364450c486696799adcc', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
trainer.tokenizer.push_to_hub('PanoEvJ/T5_base_SFT_summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'special_tokens_map.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'spiece.model'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/PanoEvJ/T5_base_SFT_summarization/commit/4e2fad682739e643fd648ce23bb6b578932fc02f', commit_message='Upload tokenizer', commit_description='', oid='4e2fad682739e643fd648ce23bb6b578932fc02f', pr_url=None, pr_revision=None, pr_num=None)